In [178]:
import random
import os
import numpy as np
import mxnet as mx

def seed_everything():
    random.seed(42)
    np.random.seed(42)
    mx.random.seed(42)
    
seed_everything()

from deeprenewal import get_dataset
from gluonts.dataset.util import to_pandas
from deeprenewal import DeepRenewalEstimator
from deeprenewal import CrostonForecastPredictor
from deeprenewal import IntermittentEvaluator
from gluonts.evaluation import Evaluator
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.trainer import Trainer
import mxnet as mx
import ast
from tqdm import tqdm
from argparse import ArgumentParser
from gluonts.model.forecast import SampleForecast
from gluonts.model.predictor import Predictor
import pandas as pd
import joblib
from pathlib import Path
from datetime import datetime

pd.set_option('display.max_columns', 500)

In [179]:
def read_data(source):
    df = pd.read_csv(source)
    
    df = df[:2322]
    
    df['Date'] = pd.to_datetime(df['pay_date'])
    df.index = df['Date']
    del df['Date'] # , df['pay_date'], 
    df.sort_index()
   
    return df

#read dataframe
df = read_data("./daily_amount.csv")
target = "amount"

In [180]:
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

def create_dataset(train_df):
    
    train_ds = ListDataset([{FieldName.TARGET: train_df[target].tolist(),
                             FieldName.START: metadata['start'],
                             FieldName.ITEM_ID: 67,
                            }],
                           freq = metadata['freq'],
                          )

    return train_ds

In [171]:
def Croston(train_ds):
    
    croston_predictor = CrostonForecastPredictor(freq= metadata['freq'], 
                                       prediction_length= metadata['prediction_length'], 
                                       variant = 'sbj', #option: original, sba, sbj
                                       no_of_params=2
                                       )

    croston_forecast = list(croston_predictor.predict(train_ds))
    
    return croston_forecast

In [148]:
metadata = {'num_series': 100,
            'num_steps': 7*4,
            'prediction_length': 4,
            'freq': '1D',
            'start': datetime.strptime("2014-11-13", "%Y-%m-%d")
           }

In [172]:
datelist = pd.date_range(datetime.strptime("2021-01-05", "%Y-%m-%d"), datetime.strptime("2021-03-22", "%Y-%m-%d")).tolist()
x = {'begin Date':[],
     'Monday':[],
     'Mon_true':[],
     'Wednesday':[],
     'Wed_true':[],
     'Friday':[],
     'Fri_true':[],
    }

for i in datelist:
    
    if i.dayofweek == 1:
        input_data = create_dataset(df.loc[:i])
        yhat = Croston(input_data)
        predict = yhat[0].samples[:, -3:-1].sum()
        true = df.loc[i+timedelta(days=2) : i+timedelta(days=3)]
        
        x['begin Date'].append(i)
        x['Monday'].append(int(predict))
        x['Mon_true'].append(sum(true['amount']))
    
    elif i.dayofweek == 3:
        input_data = create_dataset(df.loc[:i])
        yhat = Croston(input_data)
        predict = yhat[0].samples[:, -3:].sum()
        true = df.loc[i+timedelta(days=2) : i+timedelta(days=4)]
        
        x['Wednesday'].append(int(predict))
        x['Wed_true'].append(sum(true['amount']))
        
    elif i.dayofweek == 5:
        input_data = create_dataset(df.loc[:i])
        yhat = Croston(input_data)
        predict = yhat[0].samples[:, -2:].sum()
        true = df.loc[i+timedelta(days=3) : i+timedelta(days=4)]
        
        x['Friday'].append(int(predict))
        x['Fri_true'].append(sum(true['amount']))

In [175]:
result = pd.DataFrame.from_dict(x)
result

,begin Date,Monday,Mon_true,Wednesday,Wed_true,Friday,Fri_true
0,2021-01-05,3,3,5,0,3,0
1,2021-01-12,4,13,9,0,5,0
2,2021-01-19,5,1,7,6,4,9
3,2021-01-26,4,8,7,13,5,4
4,2021-02-02,5,9,6,4,5,11
5,2021-02-09,6,0,10,0,6,5
6,2021-02-16,6,6,9,5,5,12
7,2021-02-23,6,3,3,0,3,9
8,2021-03-02,4,1,1,3,1,5
9,2021-03-09,4,10,6,2,4,1


In [176]:
import math
error = []
mse = 0
for i in range(len(result)):
    a = result.loc[i, 'Monday'] + result.loc[i, 'Wednesday'] + result.loc[i, 'Friday']
    b = result.loc[i, 'Mon_true'] + result.loc[i, 'Wed_true'] + result.loc[i, 'Fri_true']
    c = (a - b)/a
    mse = mse + pow((a - b), 2)
    error.append(c)

In [177]:
result['error'] = error
print("SBJ Method MSE = ", mse/len(result))
result

SBJ Method MSE =  53.72727272727273


,begin Date,Monday,Mon_true,Wednesday,Wed_true,Friday,Fri_true,error
0,2021-01-05,3,3,5,0,3,0,0.727273
1,2021-01-12,4,13,9,0,5,0,0.277778
2,2021-01-19,5,1,7,6,4,9,0.000000
3,2021-01-26,4,8,7,13,5,4,-0.562500
4,2021-02-02,5,9,6,4,5,11,-0.500000
5,2021-02-09,6,0,10,0,6,5,0.772727
6,2021-02-16,6,6,9,5,5,12,-0.150000
7,2021-02-23,6,3,3,0,3,9,0.000000
8,2021-03-02,4,1,1,3,1,5,-0.500000
9,2021-03-09,4,10,6,2,4,1,0.071429
